In [1]:
import sys
from recommenders.datasets.python_splitters import python_chrono_split

from recommenders.datasets import covid_utils
from recommenders.models.tfidf.tfidf_utils import TfidfRecommender
from recommenders.datasets import movielens
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
import pandas as pd

System version: 3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]


In [20]:
movies_df = pd.read_csv('/home/ee303/test/dataset/GENRES/ml-1m/movies.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['movie_name', 'genre'],encoding='latin1')
#preprocess movie.csv file
movies_df.reset_index(inplace=True)
movies_df.rename(columns={"index": "itemID"}, inplace=True)

In [25]:
#applying tfidf to genres
recommender=TfidfRecommender(id_col="itemID")
clean_movies=recommender.clean_dataframe(movies_df,cols_to_clean=["genre"],new_col_name="genres").drop(columns=["genre"])
tf, vectors_tokenized = recommender.tokenize_text(df_clean=clean_movies, text_col="genres")
recommender.fit(tf, vectors_tokenized)

In [5]:
df = movielens.load_pandas_df(size="1m", local_cache_path='./dataset/')

train,validate, test = python_chrono_split(df, ratio=[0.8,0.1,0.1], filter_by="user",col_user="userID", col_item="itemID", col_timestamp="timestamp")


In [60]:
#creat rating matrix
r_matrix = df.pivot_table(index='userID', columns='itemID', values='rating')


In [53]:
#Generate recommendations based on the k most similar items to the last item rated by the user,according to timestamp,movies_df,movies info
def recommended_items(user_id,k=None):
    target_item=int(validate[validate["userID"] == user_id]["itemID"].iloc[-1])
    recommender.recommend_top_k_items(clean_movies,20)
    sim_mov=[t[1] for t in(recommender.recommendations[target_item])][:k]
    return sim_mov

In [77]:

def actual_items(user_id, rating_matrix=r_matrix):   
    user_ratings = rating_matrix.loc[user_id]
    movies_rated = user_ratings.dropna().index.tolist()
    return movies_rated 

In [85]:
def average_recall_at_k(df, k, rating_matrix=r_matrix):
    total_recall = 0
    num_users = 0
    for user_id in df['userID'].unique():
       
        recall_at_k_value = recall_at_k(user_id, k, rating_matrix)       
        total_recall += recall_at_k_value
        num_users += 1
    average_recall = total_recall / num_users if num_users > 0 else 0
    
    return average_recall

In [86]:
average_recall_at_k(test,20)